In [56]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
using CSV
using JSON
using UUIDs
using Serialization
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(dirname(pwd()) * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Simulations of Water Regulation Procedure

We would like to set up a skeleton for the day-to-day decision making of hydropower producers and the Water Regulation Company. The main steps of the Water Regulation company consist of:

* Determining a Reference Flow for the next nomination period (Inflow Forecasting)
* Receiving pre-bid nominations and calculating adjusted flow, power swaps, etc. -> Send these parameters back to the participants
* Recalculate based on renominations by power producers after bidding

For the Power Producers / Participants:  

* Fetch and Calculate all Parameters for the Water Regulation Procedure (Individual Reservoir Level etc.)
* Price and Inflow Forecasting (Fix Price Points)
* Medium-Term Hydropower Model Calculation, obtain Water Value Cuts
* Decision Model 1: Bidding and Pre-Market Clearing Nomination
* Decision Model 2: Short-Term Optimization and Renomination
* Decision Model 3: Fixed Water Flow scheduling and real time balancing

This constitutes one round of simulation.


In [57]:
filepath_Ljungan = dirname(pwd()) * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = dirname(pwd()) *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = dirname(pwd()) * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)

println("The reservoir system consists of $([r.dischargepoint for r in R])")
println("Downstream of the reservoirs we have power plants $([k.name for k in K])")
for j in J
    println("$(j.name) owns $(j.plants), which constitutes a participation rate of: \n ", j.participationrate)
end

The reservoir system consists of ["Flasjon", "Holsmjon"]


Downstream of the reservoirs we have power plants ["Flasjo", "Trangfors", "Ratan", "Turinge", "Bursnas", "Parteboda", "Hermansboda", "Ljunga", "Nederede", "Skallbole", "Matfors", "Viforsen", "Jarnvagsforsen"]
Sydkraft owns HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas], which constitutes a participation rate of: 
 Flasjon  | 1.84    
Holsmjon | 0.0     

Fortum owns HydropowerPlant[Parteboda, Hermansboda, Ljunga], which constitutes a participation rate of: 
 Flasjon  | 0.8200000000000001
Holsmjon | 0.8200000000000001

Statkraft owns HydropowerPlant[Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen], which constitutes a participation rate of: 
 Flasjon  | 1.86    
Holsmjon | 1.86    



#### Load Data

Other than the river system, the relevant data for optimization is organized in Dataframes.
We load in cleaned versions of the data.

In [58]:
filepath_prices = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Spot Prices\\prices_df.csv"
filepath_inflows = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Inflow\\Data from Flasjoen and Holmsjoen.csv"

price_data = prepare_pricedata(filepath_prices)
inflow_data = prepare_inflowdata(filepath_inflows)

first(price_data, 10)

Row,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Average,season,Weekday,CalendarWeek
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String,Int64
1,2020-12-15,17.57,17.57,16.92,16.88,18.95,20.39,22.31,24.59,31.41,31.49,31.62,30.52,35.9,38.1,40.08,43.08,37.32,51.13,47.54,40.77,35.2,28.45,23.67,21.86,30.1383,Winter,Weekday,51
2,2020-12-16,20.48,19.32,19.0,19.02,20.19,21.98,22.71,32.71,48.37,43.68,42.01,37.92,36.56,36.42,39.16,46.54,46.4,39.41,35.78,24.74,22.78,22.15,21.48,20.12,30.7887,Winter,Weekday,51
3,2020-12-17,19.03,17.93,17.53,17.28,17.97,20.0,22.09,23.64,26.79,25.28,24.43,24.3,23.67,23.99,23.97,25.06,25.38,26.07,24.22,23.16,22.19,21.97,20.76,18.99,22.3208,Winter,Weekday,51
4,2020-12-18,16.96,16.65,16.17,16.3,17.3,19.39,19.99,22.43,23.74,23.96,22.82,22.5,22.44,22.23,22.42,22.91,23.11,23.83,22.59,21.01,19.43,19.04,17.82,15.92,20.4567,Winter,Weekday,51
5,2020-12-19,16.48,15.98,15.47,15.31,15.3,15.52,15.87,15.92,16.48,17.01,17.36,17.47,17.27,17.29,17.33,17.54,17.96,18.39,17.77,17.03,15.92,15.69,15.22,13.65,16.4679,Winter,Weekend,51
6,2020-12-20,13.0,12.04,11.79,11.65,11.79,11.98,12.53,12.7,13.25,14.28,15.37,15.94,15.83,15.37,15.98,16.84,17.09,17.29,17.06,16.3,15.32,15.23,14.65,13.02,14.4292,Winter,Weekend,51
7,2020-12-21,11.9,11.2,11.5,11.73,12.81,14.37,16.67,19.74,20.6,19.92,19.32,19.39,19.44,19.2,19.45,20.65,20.55,20.35,19.49,18.03,16.17,14.47,13.18,8.66,16.6163,Winter,Weekday,52
8,2020-12-22,7.15,5.51,4.77,4.44,4.84,7.73,12.82,15.97,17.87,18.0,18.09,18.01,18.01,17.57,18.08,18.62,19.67,19.87,19.79,18.01,16.88,16.05,15.05,13.71,14.4379,Winter,Weekday,52
9,2020-12-23,13.38,13.1,12.9,12.93,13.14,14.72,15.99,18.59,19.21,19.51,18.97,19.33,19.55,19.4,19.66,19.96,20.56,21.79,20.77,18.95,17.82,16.94,16.16,15.07,17.4333,Winter,Weekday,52


### Price Forecasting

Price is sensitive to various factors. Daytime, Weekday and time of year play a relevant role in how prices usually behave.

* For the bidding and short-term optimization problems, part of the uncertainty lies in unknown hourly prices.

* For the medium term problem, we are interested in daily (average) prices.

* We want to obtain Price Points which will be important in the analysis from the historic data.

* We need penalty parameters for using the balancing markets, which could be obtained from using maximum / minimum values fromm observed spot market prices / generated scenarios.

In [72]:
const scenario_count_prices_short = 2
const scenario_count_prices_medium = 2
const stage_count_short = 2
const stage_count_medium = 52
const price_point_count = 5
const T = 24
const currentweek = 2

PriceScenariosMedium = Price_Scenarios_Medium(price_data, scenario_count_prices_medium)
PriceScenariosShort = Price_Scenarios_Short(price_data, scenario_count_prices_short, stage_count_short)
PPoints = Create_Price_Points(price_data, 10)
mu_up, mu_down = BalanceParameters(price_data)

println(PPoints)
println(mu_up," - ", mu_down)

[0.0, 24.99020833333333, 38.896645833333324, 48.37614583333334, 57.50072916666666, 69.75064583333332, 83.69897916666667, 99.48597916666667, 126.35843750000002, 159.07406250000003, 221.68049999999988, 440.0]


5.266666666666668 - 455.44


### Inflow Forecasting

Inflow has extreme seasonal differences, depending on geographical location.  
For the Ljungan River System we could observe that most inflow occurs during the spring melt around May.

In [60]:
const ColumnReservoir = Dict{Reservoir, String}(R[1] => "Flasjon Inflow", R[2] => "Holmsjon Inflow")
const scenario_count_inflows_weekly = 2
const scenario_count_inflows_short = 1


InflowScenariosShort = Inflow_Scenarios_Short(inflow_data, scenario_count_inflows_short, ColumnReservoir, R, stage_count_short, scenario_count_prices_short)
InflowScenariosMedium = Inflow_Scenarios_Medium(inflow_data, ColumnReservoir, scenario_count_inflows_weekly, R)
first(InflowScenariosShort, 10)

2-element Vector{Pair{Int64, Dict{Reservoir, Vector{Float64}}}}:
 2 => Dict(Flasjon => [8.16, 6.67], Holsmjon => [22.0, 52.5])
 1 => Dict(Flasjon => [13.53, 6.91], Holsmjon => [8.4, 2.0])

### Reference Flow

To calculate a reference flow for each reservoir, the Water Regulation Company has to combine information from inflow, historical reservoir trajectories and current reservoir level.  

In [61]:
l_traj, f = AverageReservoirLevel(R, inflow_data)
Qref = CalculateReferenceFlow(R, l_traj, f, 2)

Dict{Reservoir, Float64} with 2 entries:
  Flasjon  => 142.123
  Holsmjon => 59.9543

## Participants

Every Participant individually plans their bidding and production. Irrespective of the strategy, only limited information is available. Internal to the participants problems are 

* Price Forecasts
* Nomination to Water Regulation Company
* Bidding at Electricity Market

### Medium Term Problem

The Medium Term / Seasonal Optimization Problem is necessary to obtain Cuts for the Water Value Function.
It is read out from another file as the computation of the medium term model is computationally expensive.
Therefore the point of a functioin containing this decision model is to return cut coefficients of the Water Value Function.

* What are the inputs to the Medium Term problem?
* How are the cuts calculated?
* How many cuts should be calculated?

In [62]:
const savepath_watervalue = "C:\\Users\\lenna\\OneDrive - NTNU\\Code Master Thesis\\Inflow Forecasting\\WaterValue"

NameToParticipant = Dict{String, Participant}(j.name => j for j in J)
MediumModelDictionary_j_loaded = ReadMediumModel(savepath_watervalue * "\\Participant.jls", NameToParticipant);
MediumModelDictionary_O_loaded = ReadMediumModel(savepath_watervalue * "\\OtherParticipant.jls", NameToParticipant);


cuts_j = Dict(j => ReservoirLevelCuts(R, j.plants, j, f, currentweek, stage_count_short) for j in J)
Others = Dict(j => OtherParticipant(J,j,R)[1] for j in J)
cutsOther = Dict(j => ReservoirLevelCuts(R, Others[j].plants, Others[j], f, currentweek, stage_count_short) for j in J)
WaterCuts = Dict(j => WaterValueCuts(R, j, MediumModelDictionary_j_loaded[j], cuts_j[j], currentweek) for j in J)
WaterCutsOther = Dict(j => WaterValueCuts(R, j, MediumModelDictionary_O_loaded[j], cutsOther[j], currentweek) for j in J)

WaterCuts[J[3]]

KeyError: KeyError: key MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex, MathOptInterface.EqualTo{Float64}}(1) not found

### Create Uncertainty Set - Bidding

Depending on the Problem, we create the uncertainty sets from both price and inflow Scenarios.
This becomes more difficult for the Anticipatory Bidding Problem.

In [63]:
const iteration_count_short = 10
j = J[1]
Ω_NA, P_NA, Ω_scenario, P_scenario = create_Ω_Nonanticipatory(PriceScenariosShort, InflowScenariosShort, R, stage_count_short, scenario_count_inflows_short, scenario_count_prices_short)
Ω_A, P_A = create_Ω_Anticipatory(Ω_NA, Ω_scenario, P_scenario, J, j, R, PPoints, Qref, cutsOther, WaterCutsOther, stage_count_short, mu_up, mu_down, T)

MethodError: MethodError: no method matching create_Ω_Nonanticipatory(::Dict{Int64, Vector{Vector{Float64}}}, ::Dict{Int64, Dict{Reservoir, Vector{Float64}}}, ::Vector{Reservoir}, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  create_Ω_Nonanticipatory(!Matched::DataFrame, !Matched::DataFrame, !Matched::Int64, ::Int64, ::Int64, !Matched::Vector{Reservoir}, !Matched::Int64, !Matched::Dict{Reservoir, String})
   @ Main.WaterRegulation c:\Users\lenna\OneDrive - NTNU\Code Master Thesis\Water_Regulation\WaterRegulation.jl:1544


### Bidding Problem

* Every Power Producer solves their own Bidding problem with individual Water Value Cuts to obtain their bidding curves and nomination.
* The solutions are taken as input for the calculation of the adjusted Flow and Power Swap
* The Market clears, and the obligations for the next production day are revealed.

In [64]:
HourlyBiddingCurves = Dict{Participant, Dict{Int64, Vector{Float64}}}()
Qnoms = Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}()
Strategy = Dict{Participant, String}(j => "Nonanticipatory" for j in J)



for j in J
    if Strategy[j] == "Nonanticipatory"
        Qnom, HourlyBiddingCurve = Nonanticipatory_Bidding(R, j, PPoints, Ω_NA, P_NA, Qref, cuts_j[j], WaterCuts[j], iteration_count_short, mu_up, mu_down, T, stage_count_short)
        HourlyBiddingCurves[j] = HourlyBiddingCurve
    else
        Qnom, HourlyBiddingCurve = Anticipatory_Bidding(R, j, PPoints, Ω_A, P_A, Qref, cuts_j[j], WaterCuts[j], iteration_count_short, mu_up, mu_down, T, stage_count_short)
        HourlyBiddingCurves[j] = HourlyBiddingCurve
    end
    for r in R
        if j.participationrate[r] > 0
            Qnoms[(participant = j, reservoir = r)] = Qnom[r]
        else
            Qnoms[(participant = j, reservoir = r)] = Qref[r]
        end
    end
end


Flasjon  | 3674.0  

(e1 = 745580.6154294666, e2 = Dict(Symbol("l[Flasjon]") => 158.23861774724477))
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 2
  state variables : 296
  scenarios       : 4.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [595, 1054]
  AffExpr in MOI.EqualTo{Float64}                                               : [7, 61]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 115]
  AffExpr in MOI.LessThan{Float64}                                              : [264, 514]
  VariableRef in MOI.GreaterThan{Float64}                                    

KeyError: KeyError: key Flasjon not found

### Water Regulation

Without Updating the Reservoir Levels, the adjusted flow and Power Swaps are calculated.

In [65]:
println(Qnoms)
Qadj, _, P_Swap, _, _, _ = water_regulation(Qnoms, Qref, T, false)
println(Qadj)
println(P_Swap)

Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}()


MethodError: MethodError: no method matching Calculate_Ersmax(::Vector{Any}, ::Dict{Reservoir, Float64})

Closest candidates are:
  Calculate_Ersmax(!Matched::Array{HydropowerPlant}, ::Dict{Reservoir, Float64})
   @ Main.WaterRegulation c:\Users\lenna\OneDrive - NTNU\Code Master Thesis\Water_Regulation\WaterRegulation.jl:375


### Market Clearing

A market clearing price is generated. (From the historical prices)
For each hourly bid, the obligation is determined.

* Sample a Price for that day (Save Price)
* Perform Linear Interpolation between Price Points in which the hourly price lands, and determine the obligation from that
* Give Hourly Obligations to each user (Dictionary of Participants and hourly obligations)

In [66]:
price = Price_Scenarios_Short(price_data, 1, stage_count_short)[1][1]
Obligations = MarketClearing(price, HourlyBiddingCurves, PPoints, J, T)
println(price)
for j in J
    println(j, " - ", Obligations[j])
end

KeyError: KeyError: key Statkraft not found

### Short Term Optimization

Feed Power Swap and Nomination into Short Term Optimization Model

In [67]:
QnomO = OthersNomination(Qnoms, Qadj, J, R)

Qnoms_2 = Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}()
for j in J
    println(j)
    Qnom =ShortTermScheduling(R, j, J, Qref, Obligations[j], price, QnomO[j], Ω_NA, P_NA, cuts_j[j], WaterCuts[j], iteration_count_short, mu_up,mu_down, T, stage_count_short) 
    for r in R
        if j.participationrate[r] > 0
            Qnoms_2[(participant = j, reservoir = r)] = Qnom[r]
        else
            Qnoms_2[(participant = j, reservoir = r)] = Qref[r]
        end
    end
end

UndefVarError: UndefVarError: `Qadj` not defined

### Second Round of Water Regulation

This time, the reservoirs are updated accordingly

In [68]:
Qadj_2, _, P_Swap_2, _, _, _ = water_regulation(Qnoms_2, Qref, T, true)
println(Qadj_2)
println(P_Swap_2)
println([r.currentvolume for r in R])

UndefVarError: UndefVarError: `Qnoms_2` not defined

### Real Time Balancing and Scheduling

In [69]:
z_ups = Dict{Participant, Vector{Float64}}()
z_downs = Dict{Participant, Vector{Float64}}()
for j in J
    _, z_up, z_down, w = RealTimeBalancing(
        R,
        j,
        Qadj_2,
        P_Swap_2[j],
        T,
        mu_up,
        mu_down,
        Obligations[j])
    z_ups[j] = z_up
    z_downs[j] = z_down
end

println(z_ups)
println(z_downs)

UndefVarError: UndefVarError: `Qadj_2` not defined

### Final Revenues

From the final nominations adjusted Flow, Power Swaps, realized prices and balancing actions we can derive how much actual revenue has been generated by each user.


In [70]:
Final_Revenue(J, price, Obligations, z_ups, z_downs, mu_up, mu_down, T)


UndefVarError: UndefVarError: `Obligations` not defined

# Simulation

For a preset amount of rounds, we simulate the reservoir system through the functions and structures as described above. It is important to have a way to save results adequately! This encapsulates the calculations, and evaluations and visualizations can take place at a later point in time.